### Text Clustering Using K-means and DBSCAN

In [1]:
import numpy as np
import scipy as sp
import random
import math
from collections import defaultdict
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from scipy.sparse import csr_matrix
from sklearn.cluster import DBSCAN
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import calinski_harabaz_score
import networkx as nx

#### Reading the CSR Matrix

In [2]:
def csr_read(fname, ftype="csr", nidx=1):
    r""" 
        Read CSR matrix from a text file. 
        
        \param fname File name for CSR/CLU matrix
        \param ftype Input format. Acceptable formats are:
            - csr - Compressed sparse row
            - clu - Cluto format, i.e., CSR + header row with "nrows ncols nnz"
        \param nidx Indexing type in CSR file. What does numbering of feature IDs start with?
    """
    
    with open(fname) as f:
        lines = f.readlines()
    
    if ftype == "clu":
        p = lines[0].split()
        nrows = int(p[0])
        ncols = int(p[1])
        nnz = long(p[2])
        lines = lines[1:]
        assert(len(lines) == nrows)
    elif ftype == "csr":
        nrows = len(lines)
        ncols = 0 
        nnz = 0 
        for i in range(nrows):
            p = lines[i].split()
            if len(p) % 2 != 0:
                raise ValueError("Invalid CSR matrix. Row %d contains %d numbers." % (i, len(p)))
            nnz += len(p)/2
            for j in range(0, len(p), 2): 
                cid = int(p[j]) - nidx
                if cid+1 > ncols:
                    ncols = cid+1
    else:
        raise ValueError("Invalid sparse matrix ftype '%s'." % ftype)
    val = np.zeros(int(nnz), dtype=np.float)
    ind = np.zeros(int(nnz), dtype=np.int)
    ptr = np.zeros(nrows+1, dtype=np.long)
    n = 0 
    for i in range(nrows):
        p = lines[i].split()
        for j in range(0, len(p), 2): 
            ind[n] = int(p[j]) - nidx
            val[n] = float(p[j+1])
            n += 1
        ptr[i+1] = n 
    
    assert(n == nnz)
    
    return csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.float)

#### Calculating the IDF of the the input CSR

In [3]:
def csr_idf(matrix, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        matrix = matrix.copy()
    nrows = matrix.shape[0]
    nnz = matrix.nnz
    ind, val, ptr = matrix.indices, matrix.data, matrix.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else matrix

#### Normalising the Calculated IDF

In [4]:
def csr_l2normalize(matrix, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        matrix = matrix.copy()
    nrows = matrix.shape[0]
    nnz = matrix.nnz
    ind, val, ptr = matrix.indices, matrix.data, matrix.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = float(1.0/np.sqrt(rsum))
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return matrix

In [5]:
#Read CSR matrix from the input file
csrMatrix = csr_read('train.dat', ftype="csr", nidx=1)

#Scale the CSR matrix by idf (Inverse Document Frequency)
csrIDF = csr_idf(csrMatrix, copy=True)

#Normalize the rows of a CSR matrix by their L-2 norm.
csrL2Normalized = csr_l2normalize(csrIDF, copy=True)

#Obtain a dense ndarray representation of the CSR matrix.
denseMatrix = csrL2Normalized.toarray()

In [6]:
print(csrL2Normalized)
csrL2Normalized.shape

  (0, 13272)	0.06993666116639771
  (0, 556)	0.031789842428550756
  (0, 477)	0.05053442765379448
  (0, 956)	0.02892827625486347
  (0, 1229)	0.028083587824155785
  (0, 52)	0.03222801243394182
  (0, 54)	0.036401876151556926
  (0, 90)	0.09576566392091451
  (0, 1432)	0.03333780832574419
  (0, 3170)	0.05174109794628905
  (0, 56)	0.030100101096961172
  (0, 93)	0.030590918020923865
  (0, 94)	0.030240896982303315
  (0, 2525)	0.06240393636859998
  (0, 2842)	0.03931861920976099
  (0, 98)	0.043712293625395346
  (0, 4706)	0.05465090467007069
  (0, 2727)	0.05318919552206405
  (0, 120)	0.013815811952246083
  (0, 7093)	0.09656247105989821
  (0, 5077)	0.07014198864903001
  (0, 401)	0.05200307147257695
  (0, 4584)	0.0604208761422968
  (0, 520)	0.09921245927919291
  (0, 521)	0.2878035569692616
  :	:
  (8579, 978)	0.04197962738062193
  (8579, 1374)	0.029514731682777906
  (8579, 339)	0.010532709229759157
  (8579, 8564)	0.05900450229942135
  (8579, 5974)	0.05868349749062223
  (8579, 261)	0.00770346411573332

(8580, 126355)

In [7]:
print(denseMatrix.shape)

(8580, 126355)


#### Using the Sklearn's MiniBatch KMeans to find the initial clusters

In [8]:
kmeans = MiniBatchKMeans(n_clusters=200,random_state = 0)
kmeans.fit(csrL2Normalized)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=200, n_init=3, random_state=0, reassignment_ratio=0.01,
        tol=0.0, verbose=0)

In [9]:
label = kmeans.labels_
points =centroids= centers = kmeans.cluster_centers_

In [10]:
centers.shape, label.shape
indices = np.asarray(list(range(0,8580)))
lab = np.column_stack([indices,label])

#### My Implementation of DBSCAN

In [11]:
def MyDBSCAN(points, eps,minPts):
    neighborhoods = []
    core = []
    border = []
    noise = []

    for i in range(len(points)):
        neighbors = []
        for p in range(0, len(points)):
            # If the distance is below eps, p is a neighbor
            if sp.spatial.distance.cosine(points[i] ,points[p]) <= eps:
                neighbors.append(p)
        neighborhoods.append(neighbors)
        # If neighborhood has at least minPts, i is a core point
        if len(neighbors) >= minPts :
            core.append(i)
    # Find border points 
    for i in range(len(points)):
        neighbors = neighborhoods[i]
        # Look at points that are not core points
        if len(neighbors) < minPts:
            for j in range(len(neighbors)):
                # If one of its neighbors is a core, it is also in the core point's neighborhood, 
                # thus it is a border point rather than a noise point
                if neighbors[j] in core:
                    border.append(i)
                    # Need at least one core point...
                    break
    # Find noise points
    for i in range(len(points)):
        if i not in core and i not in border:
            noise.append(i)
            
    # # Invoke graph instance to visualize the cluster
    G = nx.Graph()
    nodes = core
    G.add_nodes_from(nodes)
    # Create neighborhood
    for i in range(len(nodes)):
        for p in range(len(nodes)):
            # If the distance is below the threshold, add a link in the graph.
            if p != i and sp.spatial.distance.cosine(points[nodes[i]] ,points[nodes[p]]) <= eps:
                G.add_edges_from([(nodes[i], nodes[p])])
    # List the connected components / clusters
    clusters = list(nx.connected_components(G))
    print("# clusters:", len(clusters))
    print("clusters: ", clusters)
    centers = []
    for cluster in clusters:
        coords = []
        for point in list(cluster):
            coords.append(points[point])
        center = np.mean(coords,axis =0)
        centers.append(center)
    expanded_clusters = clusters
    for pt in border:
        distances = {}
        for i, center in enumerate(centers):
    #         print("point = ", pt, " center = ", i)
    #         print(scipy.spatial.distance.cosine(points[pt],center))
            distances[i] = sp.spatial.distance.cosine(points[pt],center)
    #     distances = 
    #     print("closest cluster for point %d = %d " %(pt, min(distances, key=distances.get)))
        closest_cluster = min(distances, key=distances.get)
        expanded_clusters[closest_cluster].add(pt)
    #     print(clusters[closest_cluster])
    label , centroids, expanded_clusters
    centroid_labels = [len(clusters)+1]* len(centroids)
    for index, clstr in enumerate(expanded_clusters):
        for n in clstr:
            centroid_labels[n]= index
    print(np.unique(centroid_labels))
    final_labels = [0]*len(label)
    for i,l in enumerate(label):
        final_labels[i] = centroid_labels[l]
    np.unique(final_labels)
    return final_labels

In [12]:
## The number of clusters vary depending upon the epss and minPts you pass
eps =0.5
minPts = 1
d = MyDBSCAN(points, eps,minPts)
with open("submission1.txt", "w") as f:
            for l in d:
                f.write("%s\n"%l)

# clusters: 144
clusters:  [{0, 12}, {1, 41}, {2, 69, 39, 137, 52, 20, 182, 87, 185, 30}, {3}, {4}, {113, 51, 5, 31}, {6}, {7}, {98, 67, 8, 11, 142, 14, 145, 116, 59}, {104, 9, 174, 17, 181, 117}, {101, 74, 10, 108, 170, 19, 123, 62}, {13}, {15}, {16}, {25, 18, 188, 114}, {21}, {105, 141, 22}, {23}, {24, 121}, {26}, {146, 27}, {28}, {29}, {32, 144}, {33}, {34}, {35}, {36}, {37}, {38, 143}, {40}, {176, 42}, {43}, {44}, {45}, {46}, {47}, {48}, {49}, {50}, {53}, {54}, {55}, {56, 115}, {57}, {58}, {60}, {61}, {63}, {64}, {65}, {66}, {68}, {70}, {71}, {72}, {73}, {75}, {76}, {77}, {78}, {79}, {80}, {81}, {82, 151}, {83}, {84}, {130, 85}, {86}, {88}, {89}, {90}, {91, 139, 93}, {92}, {94}, {95}, {96}, {97}, {99}, {100, 191}, {102}, {103}, {106, 171}, {107}, {109}, {110}, {111}, {112}, {118}, {119}, {120, 178}, {122}, {124}, {125}, {126}, {127}, {128}, {129}, {194, 131}, {132}, {133}, {134}, {135}, {136}, {138}, {140}, {147}, {148}, {149}, {150}, {152}, {153}, {154}, {155}, {156, 183}, {157}, 